<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/wikiWord2VecCreGroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##学習済みDoc2Vecモデル
https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/

ライブラリ
Gensim: 3.6.0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from gensim.models import KeyedVectors
from pprint import pprint

#Gドライブ保存後の確認
model_dir = './drive/My Drive/NLP/wiki_tohoku.model'
model = KeyedVectors.load(model_dir)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 6.7MB/s 


In [4]:
import pandas as pd
import itertools
#**************************************************************************
#分類特定
#**************************************************************************
is_a_parent = pd.read_table('./drive/My Drive/NLP/is_a_parent_list.txt')

#dataframeをlist化
is_a_parent_list = is_a_parent.values.tolist()
#2次元listから1次元listへ
is_a_parent_list = list(itertools.chain.from_iterable(is_a_parent_list))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [0]:
import pandas as pd

def group_check(child_wd):
  max_val = 0

  val_list = []
  val_list.append(child_wd)
  
  #is-a関係の判別
  for is_a_word in is_a_parent_list:
    try:
      parent_wd = is_a_word
      isa_val_list = []

      for i in range(2):
        rvs_wd = model.most_similar(positive=[child_wd])[i][0]
        if child_wd != rvs_wd.replace('[',"").replace(']',""):
          wk_val = model.similarity(parent_wd,rvs_wd)
          isa_val_list.append(wk_val)

          for ii in range(2):
            rvs_wd2 = model.most_similar(positive=[rvs_wd])[ii][0]
            if rvs_wd != rvs_wd2.replace('[',"").replace(']',""):
              wk_val = model.similarity(parent_wd,rvs_wd2)
              isa_val_list.append(wk_val)

    except KeyError as error:
      print(error)

    #辞書に登録の無い単語の場合は対応が必要
    max_val = max(isa_val_list)
    print("【",parent_wd,"】is-a:",max_val)
    val_list.append(max_val)

  return val_list

In [0]:
import MeCab
cnt = 0

del_list =['(',')','-','"','、','。','/','.',',',':',')。',')、','「','」','『','』','()','』(','=','～','+','']

tokenizer = MeCab.Tagger("-Ochasen")
write_list = []
done_word = ''

for k, v in model.vocab.items():

  if len(k) >= 2:
    if k not in del_list:
      if k != done_word:
        node = tokenizer.parseToNode(k)
        while node:
          if node.feature.split(",")[0] == u"名詞":
            if(node.feature.split(",")[1] == u"サ変接続" or
               node.feature.split(",")[1] == u"一般" or
               node.feature.split(",")[1] == u"固有名詞"):

              #print("donewordtype:",type(done_word),done_word,len(done_word))
              #print("ktype:",type(k),k,len(k))
              
              #グループ判別
              ##rt_ave_val = group_check(k)
              #グループ判別値書き込み
              ##write_list.append(rt_ave_val)
              #print(write_list)
              cnt = cnt + 1
              done_word = k
          else:
            pass
          node = node.next

          if cnt > 5:
            break
        

In [0]:
import pandas as pd

lst = is_a_parent_list.copy()
lst.insert(0,"WORD")

df = pd.DataFrame(data=write_list,columns=lst)
df2 = df.set_index('WORD')
df2

,男性,女性,キャラクター,職業,組織,施設,都市,地域,自然,天体,道具,芸術,食べ物,乗り物,本,IT,動物,植物,身体,事件,戦争,行事,天気,病気,色,歴史,経済,言語,スポーツ,学問,現象,物,政治
WORD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [0]:
model.most_similar(positive=[u"ドイツ",u"逆"])

In [0]:
model.similarity("日本","北朝鮮")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.4340905

In [11]:
if "[日本]" not in done_list:
  print("NG")
else:
  print("OK")

OK


In [43]:
done_word

'日本'